# Syslog Resemblance Grouping

## Table of Contents

* Introduction
* Windows Log Example
    * Dataset
    * Data Processing and Sampling
    * Model Training
    * Results
        * Group Counts
        * Number of unique logs trained over
        * Number of groups identified by model training
        * Exploring group spread
* Conclusions
* References

## Introduction
SRG is designed to find a subset of representative strings within a a large collection of messages. These representative strings create groupings with which to categorize the messages for further exploration or triage.

## Windows Logs Example

### Dataset
Some example logs are available through Zenodo. This notebook will be utilizing Windows logs.


In [1]:
import urllib.request
url = 'https://zenodo.org/record/3227177/files/Windows.tar.gz'
filename = '../datasets/Windows.tar.gz'
urllib.request.urlretrieve(url, filename)

('../datasets/Windows.tar.gz', <http.client.HTTPMessage at 0x7f8ec00815b0>)

In [3]:
from subprocess import call
call(['tar', '-xzf', '../datasets/Windows.tar.gz', '-C', '../datasets/'])

0

In [10]:
import os
import sys
import glob
srg_path = os.path.abspath('../')
if srg_path not in sys.path:
    sys.path.append(srg_path)
from srg import SRG
import dask_cudf

In [11]:
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

/home/nfs/sdavis/miniconda3/envs/srg/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40635 instead
  warnings.warn(
2022-10-27 21:28:00,546 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-27 21:28:00,547 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-27 21:28:00,549 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2022-10-27 21:28:00,559 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
/home/nfs/sdavis/miniconda3/envs/srg/lib/python3.8/contextlib.py:120: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
2022-10-27 21:28:01,600 - distr

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:40635/status,
Dashboard: http://127.0.0.1:40635/status,Workers: 8
Total threads: 8,Total memory: 503.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33539,Workers: 8
Dashboard: http://127.0.0.1:40635/status,Total threads: 8
Started: Just now,Total memory: 503.79 GiB
Comm: tcp://127.0.0.1:41837,Total threads: 1
Dashboard: http://127.0.0.1:45613/status,Memory: 62.97 GiB
Nanny: tcp://127.0.0.1:44189,


### Data processing and sampling
The logs are loaded into a Dask cudf object, excess whitespace is collapsed into a single space, and then a sample of the logs is taken for training.

In [12]:
win_log_dir = '../datasets/Windows.log'
win_logs = dask_cudf.read_csv(win_log_dir, delimiter = ',', names = ['ts', 'log']).dropna()

In [13]:
def collapse_whitespace(x):
    return ' '.join(x.split())

In [14]:
def clean_up(df):
    pdf = df.to_pandas().copy()
    pdf['cleaned'] = pdf.apply(lambda row: collapse_whitespace(row['log']), axis=1)
    return pdf

In [15]:
L = win_logs.sample(frac=0.001).map_partitions(lambda df: clean_up(df))

In [16]:
len(L)

114535

### Model Training

In [17]:
win_srg = SRG()
win_srg.fit(L, column='cleaned', shingle_size=4)

Current dimension:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

### Results

In [18]:
win_labeled = win_srg.transform(L, column='cleaned')

#### Group counts

In [19]:
label_counts = win_labeled[['srg_label', 'srg_rep']].groupby('srg_rep').count().compute()
label_counts

,srg_label
srg_rep,
Info CBS Appl: DetectUpdate,169
Info CBS Appl: Evaluating applicability block(non detectUpdate part),1858
Info CBS Appl: Evaluating package applicability for package Package_34_for_KB2604115~31bf3856ad364e35~amd64~~6.1.1.3,2062
Info CBS Appl: Partial install Status testing,58
Info CBS Appl: SelfUpdate detect,45020
Info CBS Appl: detect Parent,8918
Info CBS Appl: detectParent: package: Package_8_for_KB3127220~31bf3856ad364e35~amd64~~6.1.1.0,8909
Info CBS Applicability(ComponentAnalyzerEvaluateSelfUpdate): Component: amd64_microsoft-windows-b..t-windows.resources_31bf3856ad364e35_6.1.7601.23126_ja-jp_a9f3dcbf1d3303df,26835
Info CBS Applicability(ComponentAnalyzerEvaluateSelfUpdate): Component: amd64_system.web_b03f5f7f11d50a3a_6.1.7601.18410_none_83d72e4ebeaa89e8,47


#### Number of unique logs trained over

In [20]:
len(win_labeled['cleaned'].unique())

15665

#### Number of groups identifed by model training

In [21]:
len(win_srg._groups)

51

#### Exploring group spread
The below calculates the resemblance of each log to it's representative and finds the maximum, average, and standard deviation of these resemblances for each group. This provides insight into how well each group is represented. This could also inform additional subsetting and further SRG training if there's a large, spread out group that warrants further investigation.

In [22]:
from srg.fastmap.distances import Jaccard

In [23]:
def rep_dists(x, y):
    return Jaccard(shingle_size=4).calculate(x, y)

In [24]:
def calc_dists(df):
    pdf = df.copy()
    pdf['rep_dist'] = pdf.apply(lambda row: rep_dists(row['cleaned'], row['srg_rep']), axis=1)
    return pdf

In [25]:
labeled_dists = win_labeled.map_partitions(lambda df: calc_dists(df))

In [26]:
avg_rep_dists = labeled_dists[['srg_label', 'srg_rep', 'rep_dist']].groupby(['srg_label', 'srg_rep']).agg({'rep_dist': ['count', 'max', 'mean', 'std']}).compute()

In [27]:
avg_rep_dists

rep_dist  \
                                                                count   
srg_label srg_rep                                                       
0         Info CBS Appl: SelfUpdate detect                      45020   
1         Info CBS Appl: DetectUpdate                             169   
2         Info CBS Appl: detect Parent                           8918   
3         Info CBS Appl: Partial install Status testing            58   
4         Info CBS EvaluateApplicability                         1729   
5         Info CSI 000007e5 Transaction merge required             34   
6         pA = PROCESSOR_ARCHITECTURE_AMD64 (9)                    18   
7         Version = 11.2.9600.17914                                13   
8         Version = 11.2                                            4   
9         Version = 6.2.7601.18514                                  3   
10        Version = 6.1.7600.16385                                 20   
11        Version = 6.1.7601.18489                                 20   
12        Version = 6.1.7601.23542                                 31   
13        Info CBS Appl: Evaluating applicability block(n...     1858   
14        Info CBS Session: 30547927_1113567448 initializ...      672   
15        Info CBS Appl: detectParent: package: Package_8...     8909   
16        Info CBS Plan: Package: Package_for_KB3003743_S...      189   
17        Info CBS Appl: Package: Microsoft-Windows-IIS-W...        3   
18        Info CBS Exec: Installing Package: Microsoft-Wi...        2   
19        Info CBS Failed to evaluate external applicabil...        2   
20        Info CBS Appl: Evaluating package applicability...     2062   
21        Info CBS Appl: detectParent: package: Microsoft...       83   
22        Info CSI 000014dc Unable to prestage file [l:62...        2   
23        Info CBS Failed to get session package state fo...        8   
24        Info CBS Failed to get session package state fo...       46   
25        Info CBS Failed to get session package state fo...       86   
26        Info CBS Failed to get session package state fo...       26   
27        Info CBS Read out cached package applicability ...        2   
28        Info CBS Plan: Skipping package since its start...        2   
29        Info CBS Appl: Evaluating package applicability...        1   
30        Info CBS Read out cached package applicability ...       20   
31        Info CBS Applicability(ComponentAnalyzerEvaluat...       52   
32        Info CBS Applicability(ComponentAnalyzerEvaluat...    26835   
33        Info CBS Applicability(ComponentAnalyzerEvaluat...    16916   
34        Info CBS Applicability(ComponentAnalyzerEvaluat...      255   
35        Info CBS Applicability(ComponentAnalyzerEvaluat...        2   
36        Info CBS Applicability(ComponentAnalyzerEvaluat...       27   
37        Info CBS Applicability(ComponentAnalyzerEvaluat...        6   
38        Info CBS Applicability(ComponentAnalyzerEvaluat...       25   
39        Info CBS Applicability(ComponentAnalyzerEvaluat...       47   
40        Info CBS Applicability(ComponentAnalyzerEvaluat...       19   
41        Info CBS Applicability(ComponentAnalyzerEvaluat...       49   
42        Info CBS Applicability(ComponentAnalyzerEvaluat...       31   
43        Info CBS Applicability(ComponentAnalyzerEvaluat...       15   
44        Info CBS Unloading offline registry hive: {bf1a...        3   
45        Info CSI 000005f0@2017/4/18:19:39:17.607 CSI Tr...       25   
46        Info CSI 0000020d@2015/8/30:19:01:02.543 CSI Tr...        2   
47        Info CBS WatchList: Add package Package_92_for_...      211   
49        Info CSI 0000002c Processing Namespace uninstal...        3   
48        Info CSI 000004bb@2016/12/28:19:10:48.572 CSI T...        1   
50        Info CSI blicKey neutral]) ref: ( flgs: 0000000...        1   

                                                                        \
                                            

### Conclusions
This shows how logs (or any string data) can be sytactically grouped with minimal *a priori* parameterization. After fitting the model to your data, group statistics can be calculated for dataset insights or specific groups of interest can be further explored for triage and informed filtering. Models can be saved for future inferencing.

### References
https://github.com/logpai/loghub